In [10]:
from Def import*

In [ ]:
for index, row in df.iterrows():
    if row['signal']=='buy':
        print('Buy',row['close'])
    elif row['signal']=='sell':
        print('Sell',row['close'])

In [1]:
purchase_price = 28049
purchase_amount = 1000
commission = 0.0001
sell_price = 28165

# Рассчитываем количество купленных биткоинов
bitcoins_bought = (purchase_amount * (1 - commission)) / purchase_price

# Рассчитываем сумму, полученную после продажи биткоинов
sell_amount = bitcoins_bought * sell_price * (1 - commission)

# Рассчитываем прибыль
profit = sell_amount - purchase_amount

print("Прибыль: $", format(profit, ".2f"))

Прибыль: $ 3.93


In [69]:
purchase_price = 27608.5
purchase_amount = 1000
maker_fee_rate = 0.0010
taker_fee_rate = 0.0010

# Расчет количества купленных биткоинов
bitcoins_bought = purchase_amount / purchase_price

# Расчет комиссии мейкера и тейкера в биткоинах
maker_fee = bitcoins_bought * maker_fee_rate
taker_fee = bitcoins_bought * taker_fee_rate

# Конвертация комиссии из биткоинов в USDT по текущей цене
maker_fee_usdt = maker_fee * purchase_price
taker_fee_usdt = taker_fee * purchase_price

print("Комиссия мейкера на покупку: ", round(maker_fee_usdt, 4), "USDT")
print("Комиссия тейкера на покупку: ", round(taker_fee_usdt, 4), "USDT")

# Расчет комиссии мейкера и тейкера на продажу
sell_price = purchase_price

# Расчет комиссии мейкера и тейкера в биткоинах
maker_fee_sell = bitcoins_bought * maker_fee_rate
taker_fee_sell = bitcoins_bought * taker_fee_rate

# Конвертация комиссии из биткоинов в USDT по текущей цене
maker_fee_sell_usdt = maker_fee_sell * sell_price
taker_fee_sell_usdt = taker_fee_sell * sell_price

print("Комиссия мейкера на продажу: ", round(maker_fee_sell_usdt, 4), "USDT")
print("Комиссия тейкера на продажу: ", round(taker_fee_sell_usdt, 4), "USDT")


Комиссия мейкера на покупку:  1.0 USDT
Комиссия тейкера на покупку:  1.0 USDT
Комиссия мейкера на продажу:  1.0 USDT
Комиссия тейкера на продажу:  1.0 USDT


In [29]:
import pandas as pd

def moving_average_difference(df, periods):
    column_name = f'diff_close{periods}'
    total_difference = 0

    for i in range(1, periods + 1):
        total_difference += (df['close'] - df['close'].shift(i))

    df[column_name] = (total_difference / periods).round(2)
    return df

# Пример использования функции
data = {'close': [1000, 1050, 1100, 1150, 1200, 1250, 1300, 1350]}
df = pd.DataFrame(data)

periods = 2
df = moving_average_difference(df, periods)
print(df)


   close  diff_close2
0   1000          NaN
1   1050          NaN
2   1100         75.0
3   1150         75.0
4   1200         75.0
5   1250         75.0
6   1300         75.0
7   1350         75.0


In [ ]:
df['diff_close4'] = ((df['close'] - df['close'].shift(4)) + (df['close'] - df['close'].shift(3))+(df['close'] - df['close'].shift(2))+(df['close'] - df['close'].shift(1)))/4

In [40]:
def moving_average_difference(df, periods):
    column_name = f'diff_close{periods}'

    # Рассчитываем процентное изменение между текущим и предыдущим значением
    df[column_name] = ((df['close'] / df['close'].shift(1)) - 1) * 100

    # Рассчитываем скользящее среднее
    df[column_name] = df[column_name].rolling(periods).mean().round(2)

    return df


# Пример использования функции
data = {'close': [1000, 1050, 1100, 1150, 1200, 1250, 1300, 1350]}
df = pd.DataFrame(data)

periods = 2
df = moving_average_difference(df, periods)
print(df)


   close  diff_close2
0   1000          NaN
1   1050          NaN
2   1100         4.88
3   1150         4.65
4   1200         4.45
5   1250         4.26
6   1300         4.08
7   1350         3.92


In [ ]:
# Устанавливаем начальную сумму денег и количество криптовалют, которые у нас есть
initial_cash = 1000
initial_cash2 = initial_cash
initial_coins = 0

# Создаем список для хранения изменений стоимости криптовалют
coin_prices = []

# Устанавливаем комиссию в 0.1%
commission = 0.0001

# Инициализируем счетчики для статистики
total_trades = 0
successful_trades = 0
unsuccessful_trades = 0

# Проходим по строкам таблицы
for index, row in df.iterrows():
    # Если есть позиция на покупку
    if position == 'long':
        # Проверяем, если есть сигнал на продажу
        if row['signal'] == 'sell':
            # Продаем криптовалюты
            coins_sold = initial_coins
            coin_prices.append(row['close'])
            initial_cash += coins_sold * row['close'] * (1 - commission)
            initial_coins = 0
            position = 'out'

            # Обновляем счетчики статистики
            total_trades += 1
            if len(coin_prices) > 1 and coin_prices[-1] > coin_prices[-2]:
                successful_trades += 1
            else:
                unsuccessful_trades += 1

    # Если нет позиции на покупку, но есть сигнал на покупку
    elif position == 'out' and row['signal'] == 'buy':
        # Покупаем криптовалюты
        coins_bought = (initial_cash * (1 - commission)) / row['close']
        coin_prices.append(row['close'])
        initial_coins += coins_bought
        initial_cash = 0
        position = 'long'

# Если остались криптовалюты в конце, продаем их по последней известной цене
if initial_coins > 0:
    coins_sold = initial_coins
    coin_prices.append(df.iloc[-1]['close'])
    initial_cash += coins_sold * df.iloc[-1]['close'] * (1 - commission)
    initial_coins = 0
    position = 'out'

# Вычисляем изменение суммы денег
profit = initial_cash - initial_cash2

# Вычисляем win rate
win_rate = (successful_trades / total_trades) * 100 if total_trades > 0 else 0

# Выводим результаты
print("initial_cash2:", initial_cash2)
print("Profit: $", profit)
print("Total Trades:", total_trades)
print("Successful Trades:", successful_trades)
print("Unsuccessful Trades:", unsuccessful_trades)
print("Win Rate: {:.2f}%".format(win_rate))
print("commission:",commission)
print("days:",days)